In [1]:
from pathlib import Path
import pandas as pd
import xlwings as xw
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Access input folder
input_dir1 = Path ("C:/Users/z5379606/OneDrive - UNSW/Viperblast/Reflected_pressure/Near_field_time_history_txt_files_dummy")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

# Create output directory
output_dir1 = Path("C:/Users/z5379606/OneDrive - UNSW/Viperblast/Reflected_pressure/Near_field_time_history_dummy")
output_dir1.mkdir(exist_ok = True)

# Define current work directory
current_dir = Path.cwd()

# Access the template to be used to generate files
excel_template = current_dir/ "Template.xlsx"

1 C:\Users\z5379606\OneDrive - UNSW\Viperblast\Reflected_pressure\Near_field_time_history_txt_files_dummy
2 [WindowsPath('C:/Users/z5379606/OneDrive - UNSW/Viperblast/Reflected_pressure/Near_field_time_history_txt_files_dummy/1.25m'), WindowsPath('C:/Users/z5379606/OneDrive - UNSW/Viperblast/Reflected_pressure/Near_field_time_history_txt_files_dummy/2.75m')]


In [3]:
A_mapping = {1: 0, 2: 15, 3: 30, 4: 45}
df_all = pd.DataFrame(columns=["Mass", "Standoff distance", "Angle", "Arrival time", "AP", "Change time", "CP", "Trough time", "TP", "End time", "EP", "Status"])

for folder1 in input_dir2: # folder1-1.00m

    input_dir3 =  [folder2 for folder2 in folder1.iterdir() if folder2.is_dir()]
    # Create output folders
    output_dir2 = output_dir1/ folder1.name
    output_dir2.mkdir(exist_ok = True)
    print ("OUTPUT 2", output_dir2)
    
    for folder2 in input_dir3: # folder2-1.00m00.5kg
        # SD AND EM
        SD = float(folder2.name[0:4])
        EM = float(folder2.name[5:9])
        
        # Create output folders
        output_dir3 = output_dir2/ folder2.name
        output_dir3.mkdir(exist_ok = True)
        print ("OUTPUT 3", output_dir3)
        
        # Make a list of data file names
        txt_files = list(folder2.rglob("*.txt"))
        file1 = txt_files[0]
        print (file1)
        
        # Read data from Excel file
        df1 = pd.read_csv(file1, delimiter='\s+', header=None) 
        
        # Initiate xlwings library
        with xw.App (visible = False) as app:
        
            for i in range (1,5):
                # A
                A = A_mapping.get(i, None) 
                
                time = df1.iloc[:,0].dropna().values*1000
                pressure = df1.iloc[:,i].dropna().values/1000
                
                wb = app.books.open(excel_template)
                
                peak_index = np.argmax(pressure)
                # ARRIVAL
                AT = time[peak_index]
                AP = pressure[peak_index]
                
                pressure = pressure[peak_index:]
                time = time[peak_index:]
                
                lowest_index = np.argmin(pressure[:])
                # TROUGH
                TT = time[lowest_index]
                TP = pressure[lowest_index]
                
                if pressure[lowest_index] < -0.2:
                    Status = "if"
                    
                    # Find the first zero crossing between peak and lowest value
                    zero_crossing_index_1 = next((i for i, val in enumerate(pressure[:lowest_index], start=0) if val <= 0), None)
                
                    zero_crossing_index_2 = next((i for i, val in enumerate(pressure[lowest_index:], start=lowest_index) if val > -0.2), None)
                    #zero_crossing_index_2 = next((i for i, val in enumerate(smoothed_pressure[lowest_index:], start=lowest_index) if val > -0.1), None)
                    
                    if zero_crossing_index_2 is None:
                        Status = "if_if"
                        zero_crossing_index_2 = min(range(lowest_index, len(pressure)),key=lambda i: abs(pressure[i]))
                        
                elif -0.2 < pressure[lowest_index] < 0:
                    Status = "elif"
                    # Find the first zero crossing between peak and lowest value
                    zero_crossing_index_1 = next((i for i, val in enumerate(pressure[:lowest_index], start=0) if val <= 0), None) 
                    
                    zero_crossing_index_2 = next((i for i, val in enumerate(pressure[lowest_index:], start=lowest_index) if val > -0.05), None)
                    
                else:
                    Status = "else"
                    zero_crossing_index_1 = lowest_index
                    zero_crossing_index_2 = lowest_index
                # CHANGE AND END
                CT = time[zero_crossing_index_1]
                CP = pressure[zero_crossing_index_1]
                ET = time[zero_crossing_index_2]
                EP = pressure[zero_crossing_index_2]
                
                print (peak_index, zero_crossing_index_1, lowest_index, zero_crossing_index_2)

                # Creating a new DataFrame to append
                row_all = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A],
                                        "Arrival time": [AT],"AP": [AP],
                                        "Change time": [CT], "CP": [CP],
                                        "Trough time": [TT], "TP": [TP],
                                        "End time": [ET], "EP": [EP],
                                        "Status": [Status]})
                df_all = pd.concat([df_all, row_all], ignore_index=True)
                
                # Create a dictionary with the arrays
                data1 = {"Mass": [EM] * len(time[: zero_crossing_index_2+1]),
                         "Standoff distance": [SD] * len(time[: zero_crossing_index_2+1]),
                         "Angle": [A] * len(time[: zero_crossing_index_2+1]),
                         'Time': time[: zero_crossing_index_2+1],
                         'Pressure': pressure[: zero_crossing_index_2+1]}
                    
                # Create DataFrames from dictionaries
                df2 = pd.DataFrame(data1)
                    
                # Write dataframe in excel template
                wb.sheets[0].range("A1").options(index=False).value = df2
                
                # Create files in output directory
                wb.save(output_dir1/f"{i}.xlsx")
                    
                print ("file_name",i)
                
                # Create the plot
                plt.figure(figsize=(4, 4))
                plt.plot(time, pressure, color='b', linestyle=':')
                plt.plot(time[: zero_crossing_index_2+1], pressure[: zero_crossing_index_2+1], color='r', linestyle='-')
                # Save the plot as an imageadd
                plt.savefig(output_dir3/f'{i}.png', dpi=150, bbox_inches='tight', facecolor='white')
                    
                # Close the plot to free up memory
                plt.close()
                    
                # Create files in output directory
                wb.save(output_dir1/f"{i}.xlsx")

OUTPUT 2 C:\Users\z5379606\OneDrive - UNSW\Viperblast\Reflected_pressure\Near_field_time_history_dummy\1.25m
OUTPUT 3 C:\Users\z5379606\OneDrive - UNSW\Viperblast\Reflected_pressure\Near_field_time_history_dummy\1.25m\1.25m02.5kg
C:\Users\z5379606\OneDrive - UNSW\Viperblast\Reflected_pressure\Near_field_time_history_txt_files_dummy\1.25m\1.25m02.5kg\viper3d_th_overpressure.txt
34 573 1201 1397


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', 'SaveAs method of Workbook class failed', 'xlmain11.chm', 0, -2146827284), None)

In [ ]:
for folder1 in input_dir2: # folder1-1.00m

    input_dir3 =  [folder2 for folder2 in folder1.iterdir() if folder2.is_dir()]
    # Create output folders
    output_dir2 = output_dir1/ folder1.name
    output_dir2.mkdir(exist_ok = True)
    #print ("OUTPUT 2", output_dir2)
    
    for folder2 in input_dir3: # folder2-1.00m00.5kg
        # SD AND EM
        SD = float(folder2.name[0:4])
        EM = float(folder2.name[5:9])
        
        # Create output folders
        output_dir3 = output_dir2/ folder2.name
        output_dir3.mkdir(exist_ok = True)
        #print ("OUTPUT 3", output_dir3)
        
        # Make a list of data file names
        txt_files = list(folder2.rglob("*.txt"))
        file1 = txt_files[0]
        #print (file1)
        
        # Read data from Excel file
        df1 = pd.read_csv(file1, delimiter='\s', header=None) 
        print (df1.iloc[0,0])

In [ ]:
# Write dataframes in excel template
all_path = 'RP_near_field_all.xlsx'

df_all.to_excel(all_path, index=False)